In [ ]:
import os
import sys
from pathlib import Path
from dotenv import load_dotenv

load_dotenv()
pythonpath = str(Path(os.getenv("PYTHONPATH")))
sys.path.insert(0, pythonpath)

In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import yaml
import joblib
import json
from pprint import pprint

# does not really work and / or make sense if 
from src.report.visualization import plot_confusion_matrix

# 1 Config

In [ ]:
with open("../params.yaml", "r") as conf_file:
    config = yaml.safe_load(conf_file)

pprint(config)

# 2 Load dataset

In [ ]:
# Get data 

import pandas as pd
from sklearn.datasets import load_iris

data = load_iris(as_frame=True)
dataset = data.frame
dataset.head()

In [ ]:
# print labels for target values 

[print(f'{target}: {label}') for target, label in zip(data.target.unique(), data.target_names)]

In [ ]:
# feature names

dataset.columns = [colname.strip(' (cm)').replace(' ', '_') for colname in dataset.columns.tolist()]

feature_names = dataset.columns.tolist()[:4]
feature_names

In [ ]:
dataset.to_csv(config['data']['dataset_csv'], index=False)

# 3 Features engineering

In [ ]:
dataset['sepal_length_to_sepal_width'] = dataset['sepal_length'] / dataset['sepal_width']
dataset['petal_length_to_petal_width'] = dataset['petal_length'] / dataset['petal_width']

dataset = dataset[[
    'sepal_length', 'sepal_width', 'petal_length', 'petal_width',
#     'sepal_length_in_square', 'sepal_width_in_square', 'petal_length_in_square', 'petal_width_in_square',
    'sepal_length_to_sepal_width', 'petal_length_to_petal_width',
    'target'
]]

In [ ]:
dataset.head()

In [ ]:
# Save features
dataset.to_csv(config['data']['features_path'], index=False)

# 4 Split dataset

In [ ]:
test_size=0.2

## Splittail train/test

In [ ]:
train_dataset, test_dataset = train_test_split(dataset, test_size=test_size, random_state=42)
train_dataset.shape, test_dataset.shape 

train_dataset.to_csv(config['train']['trainset_path'])
test_dataset.to_csv(config['train']['testset_path'])

# 5 Train

In [ ]:
# Get X and Y

y_train = train_dataset.loc[:, 'target'].values.astype('int32')
X_train = train_dataset.drop('target', axis=1).values.astype('float32')

In [ ]:
# Create an instance of Logistic Regression Classifier CV and fit the data

logreg = LogisticRegression(C=0.001, solver='lbfgs', multi_class='multinomial', max_iter=100)
logreg.fit(X_train, y_train)

In [ ]:
joblib.dump(logreg, config['train']['model_path'])

# 6 Evaluate

In [ ]:
# Get X and Y
y_test = test_dataset.loc[:, 'target'].values.astype('int32')
X_test = test_dataset.drop('target', axis=1).values.astype('float32')

In [ ]:
prediction = logreg.predict(X_test)
cm = confusion_matrix(prediction, y_test)
f1 = f1_score(y_true = y_test, y_pred = prediction, average='macro')

In [ ]:
# f1 score value
f1

In [ ]:
# Save metrics
metrics = {
    'f1': f1
}

with open(config['reports']['metrics_file'], 'w') as mf:
    json.dump(
        obj=metrics,
        fp=mf,
        indent=4
    )

In [ ]:
cm_plot = plot_confusion_matrix(cm, data.target_names, normalize=False)
cm_plot

In [ ]:
# Save confusion matrix image
cm_plot.savefig(config['reports']['confusion_matrix_image'])